In [72]:
import os
from PIL import Image
import numpy as np
from torchvision.models import mobilenet_v3_large
import torch.nn as nn
from torchvision import transforms
import matplotlib.pyplot as plt
import torch
import shutil
from classifier import ImageClassifier
import glob
import cv2

In [73]:
model = mobilenet_v3_large()
    
modules = list(model.children())[:-1]  # delete the last fc layer.
model = nn.Sequential(*modules)

model.classifier = nn.Sequential(nn.Flatten(),
                                nn.Linear(960, 4096),
                                nn.ReLU(),
                                nn.Linear(4096, 4096),
                                nn.ReLU(),
                                nn.Linear(4096, 1000),
                                nn.ReLU(),
                                nn.Dropout(p=0.5),
                                nn.Linear(1000, 2),
                                nn.Softmax(dim=1)
                            )

In [74]:
# Load the model
ckpt_path = "best_acc_epoch=168-val_acc=0.96.ckpt"
best_model = ImageClassifier.load_from_checkpoint(checkpoint_path=ckpt_path, model=model)

In [75]:
transform = transforms.Compose([transforms.Resize(size=(224,224)),
        transforms.ToTensor(), transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

In [76]:
def find_coords(input_array): # finds the coordinates to crop rgb image
    bottom = input_array.nonzero()[0].max()
    top = input_array.nonzero()[0].min()
    
    right = input_array.nonzero()[1].max()
    left = input_array.nonzero()[1].min()
    
    return left, top, right, bottom

In [77]:
def run_model(img): # runs the image through classification model
    best_model.eval()
    with torch.no_grad():
        transformed_img = img.unsqueeze(dim=0)
        output = best_model(transformed_img)
    pred_label = torch.argmax(output, dim=1)
    
    return(pred_label.item())

In [78]:
def save_to_folder(pred, mask_path, person_path): # saves person and mask to desired folder
    mask_id = mask_path[18:]
    person_id = person_path[18:]
    
    if pred == 0:
        dest_mask = f'.\\output\\invalid\\{mask_id}'
        dest_person = f'.\\output\\invalid\\{person_id}'
    else:
        dest_mask = f'.\\output\\valid\\{mask_id}'
        dest_person = f'.\\output\\valid\\{person_id}'
    
    # create a copy of the mask PNG and person PNG to the respective folder
    shutil.copy(mask_path, dest_mask)
    shutil.copy(person_path, dest_person)

In [79]:
files = glob.glob('pedestrian_pool\\*_mask*')

In [81]:
count=0
for i in range(len(files)):
    mask = files[i]
    rgb = files[i].replace('mask', 'person')
    """# check to make sure mask and rgb files match
    if mask[:-9] == rgb[:-11]:"""
    mask_img = Image.open(mask)
    mask_array = np.array(mask_img)
    
    # check if mask has over 5000 pixels
    if np.sum(mask_array == 1) > 5000:
        # find number of contours of each mask
        mask_cv2 = cv2.imread(mask)
        gray = cv2.cvtColor(np.array(mask_cv2), cv2.COLOR_BGR2GRAY)
        contours, _ = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 
        
        if len(contours) == 1:

            rgb_img = Image.open(rgb)
            rgb_img = rgb_img.crop((find_coords(mask_array)))
    
            # apply transformation and convert to tensor
            img = transform(rgb_img)
            # evaluate using best model
            pred = run_model(img) 
            # save mask to respective folder
            save_to_folder(pred, mask, rgb)
        else:
            count += 1
    else:
        count+=1

In [82]:
print(f'# of Images <= 1000 Pixels: {count}')
print(f'# of Images > 1000 Pixels: {len(files)-count}')

# of Images <= 1000 Pixels: 6458
# of Images > 1000 Pixels: 1192
